# Sensorfusion - Autonomes Fahren


In [91]:
# Imports
import pandas as pd
from pyds import MassFunction


In [92]:
data=pd.read_csv("DPE16_1.csv", sep=';', decimal=",")

In [93]:
data.head()

,t,Eigengeschwindigkeit(m/s),Bbox_Breite,Bbox_Hoehe,Bbox_links_unten_y,Abstand(m)
0,0.0,22.0,90,99,153,76.5
1,0.1,22.2,90,100,154,NaN
2,0.2,22.2,86,95,154,77.0
3,0.3,22.1,90,100,153,76.5
4,0.4,22.1,90,100,153,76.5


# Notizen
Jede Zeile entspricht Messung.
Verhältnis Breite/Höhe durch BBox-Werte.
Geschwindigkeit nur über Zeit (min Anzahl an Messwerten?) durch Abstand Differenz - Eigengeschwindigkeit
Beschleunigung über Zeit (sobald 2 Werte für Geschwindigkeit vorhanden: GeschwindigkeitNeu - GeschwindigkeitAlt

Konfidenz höher, je kleiner der Abstand
Omega: 1 - Konfidenz
omega: P L M F



In [94]:
omega = 'plmf'
#Save speed of previous measurement
oldSpeedSaved = None
newSpeedSaved = 0


In [95]:
#def calcBoundingRatio(oldRatio, newRatio):
    
#catch nullvalue
def calcSpeed(oldOwnSpeed, newOwnSpeed, oldDistance, newDistance):
    difDistance = newDistance - oldDistance
    #Use average speed because old and new speed can be slightly different
    ownSpeed = (oldOwnSpeed + newOwnSpeed) / 2
    speed = difDistance / 0.1 + ownSpeed
    saveSpeed(speed)
    
    return speed
    
# Manage saved values of speed to calculate acceleration
def saveSpeed(speed):
    if (newSpeedSaved):
        oldSpeedSaved = newSpeedSaved
    newSpeedSaved = speed
    
#Catch oldSpeed null
def calcAcceleration():
    if (oldSpeedSaved == None):
        print('Erste Messung, noch keine Daten zur Beschleunigung vorhanden')
        # Return 0 by guessing, two measurements are required.
        # No problem because acceleration will only have effect if it is high
        return 0
    speedDif = newSpeedSaved - oldSpeedSaved
    acc = speedDif / 0.1
    
    return acc
    
#def calcMasses(oldMeasurement, newMeasurement):
    # Call other calcs
    
testspd1 = calcSpeed(22.2, 22.1, 77.0, 76.5)
testspd2 = calcSpeed(22.1, 22.1, 76.5, 76.5)

print(testspd1)
print(testspd2)

testacc = calcAcceleration()
print(testacc)

UnboundLocalError: local variable 'newSpeedSaved' referenced before assignment

In [ ]:

def iterateMeasurement():
    
    